# [Exploration 17] Session-Based Recommendation

Session-Based 추천 모델은 전자 상거래 플랫폼에서 사용자의 행동을 분석해서 '지금 시점에 고객이 구매할 만한 물건을 추천'해주는 예측 모델이다. 

In [14]:
import datetime as dt
from pathlib import Path
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [15]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose-data/ml-1m') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


## Step 1. 데이터의 전처리

가장 먼저 유저수(UserId)와 아이템 수(ItemId)를 확인해본다.

In [16]:
data['UserId'].nunique(), data['ItemId'].nunique()

(6040, 3706)

6040명의 유저와 3706개의 아이템이 있다.

In [17]:
min = data['Time'].min()
max = data['Time'].max()

sec = max - min
minutes = sec / 60
hour = minutes / 60
day = hour / 24

day / 365

2.8459746955859972

대략 2년하고 두달정도의 기간동안의 각 유저별로 구매한 item의 종류와 수를 파악하는 데이터인것 같다.   
#### 1970년 1월 1일을 기준으로 초단위가 지나갔다고 했으니 1972년 3월까지의 데이터로 파악할 수 있다.   
1990년에 인터넷 쇼핑이라는 개념이 도입된걸 감안하면 오프라인 데이터를 정리한 데이터셋이 아닐까 싶다.

In [18]:
session_length = data.groupby('UserId').size()
data['session_length'] = session_length #session_length라는 컬럼을 하나 더 만들어준다.

그렇다면 session_length에 대해 다시 생각해볼 수 있다. 기간을 몇일을 기준으로 생각해야 적합한 추천이 이루어질 수 있을까?   
2년동안 최소 20개를 구매한 사람이 있고, 2314개를 구매한 사람이 있다. 

In [19]:
data.describe()

,UserId,ItemId,Rating,Time,session_length
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06,6040.000000
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08,165.597517
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07,192.747029
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08,20.000000
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08,44.000000
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08,96.000000
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08,208.000000
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09,2314.000000


75%이상의 사람들은 208개 정도를 구매했으며, 평균적으로 165개를 구매했다.

In [20]:
length_count = session_length.groupby(session_length).size()
length_percent_cumsum = length_count.cumsum() / length_count.sum()
length_percent_cumsum_999 = length_percent_cumsum[length_percent_cumsum < 0.999]

length_percent_cumsum_999

20      0.014238
21      0.029305
22      0.042053
23      0.055464
24      0.068874
          ...   
1271    0.998179
1277    0.998344
1286    0.998510
1302    0.998675
1323    0.998841
Length: 736, dtype: float64

또 한가지 생각해볼 문제로는 ratings이다. 1, 2점으로 평가된 제품을 다른 사람에게 추천해줄 수 없으므로 이 또한 제외시키도록 하겠다.

In [21]:
rating_filter = data.Rating > 2
data = data.loc[rating_filter, :]

In [22]:
data['Rating'].min()

3

In [23]:
# year = 60*60*24*30*12 #60초*60분*24시간*30일*12달
# year

31104000

기간은 1년을 기준으로 자르고, 마지막 20일째는 validation set, 마지막 10일째는 test set으로 설정하겠다.

In [24]:
# sorted_time = data['Time'].max() - year
# data = data[data['Time'] > sorted_time]

# data

In [25]:
# month_ago = latest - dt.timedelta(30)     # 최종 날짜로부터 30일 이전 날짜를 구한다.  
# data = data[data['Time'] > month_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
# data

In [26]:
data

,UserId,ItemId,Rating,Time,session_length
31,1,3186,4,978300019,119.0
22,1,1270,5,978300055,297.0
27,1,1721,4,978300055,70.0
37,1,1022,5,978300055,53.0
24,1,2340,3,978300103,136.0
...,...,...,...,...,...
1000019,6040,2917,4,997454429,NaN
999988,6040,1921,4,997454464,NaN
1000172,6040,1784,3,997454464,NaN
1000167,6040,161,3,997454486,NaN


날짜를 기준으로 기간을 조절하려 했으나 데이터가 너무 적어져 기간을 따로 설정하지 않고 진행하겠다.

In [27]:
data = data.sort_values(by=['Time'])

In [45]:
# del data['session_length']
data.sort_values(by=['UserId'])

,UserId,ItemId,Rating,Time
22,1,1270,5,978300055
18,1,3105,5,978301713
40,1,1,5,978824268
48,1,2028,5,978301619
41,1,1961,5,978301590
...,...,...,...,...
1000079,6040,1283,4,960971931
1000053,6040,1244,4,960971948
1000067,6040,722,3,960971992
1000112,6040,2645,5,960971797


In [46]:
userid = data['UserId']
rating = data['Rating']
itemid = data['ItemId']

In [47]:
data1 = pd.merge(userid, rating)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

이번 자료에서 사용할 모델은 2016년 ICLR에 공개된 SESSION-BASED RECOMMENDATIONS WITH RECURRENT NEURAL NETWORKS 입니다.


Session Data에서는 처음으로 RNN 계열 모델을 적용하여 발표했습니다.

 

모델 구조는 아래 그림처럼 간단합니다.

 

여러 RNN 계열의 모델(e.g. LSTM)이 있겠지만 저자가 실험해본 결과 GRU의 성능이 제일 좋았다고 합니다.


또한 Embedding Layer를 사용하지 않았을 때가 사용했을 때보다 성능이 좋았다고 합니다.


이번 자료에서도 Embedding Layer를 사용하지 않고 One-hot Encoding만 사용하겠습니다.